In [10]:
import pandas as pd
# location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
# csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

# csv_file.index = csv_file.columns
# string_to_remove = "_nostop.fas.aligned.fas.best"
# csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
# csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

# string_to_remove = ".fas.ClipKIT"
# csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
# csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


# matrix_long = csv_file.reset_index().melt(
#     id_vars="index",         # Keeps row names as a column
#     var_name="Gene1",       # New name for columns
#     value_name="Gene2"       # New name for matrix values
# )

# # Rename 'index' to 'Row' for clarity
# diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/g/My Drive/Circadian Rhythm Genes Project/19.Evolutionary_rate_co-variation(ERC)/2.Two_state"
print("AA")
diapause = pd.read_csv(f"{location}/ERC_correlation_diapause_long.csv")
# diapause = diapause.sample(n=10, random_state=42)
location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

diapause_rn["Correlation"].fillna(1, inplace=True)

# print(diapause_rn)
print("AA")
genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
# diapause_rn["Correlation"] += 1
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK","1422at7088","5474at7088","818at7088"]

# ### For annotated tree
# for row in df_filtered.itertuples():
#     if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
#         if row.Annotation_gene1 not in gene_dictionary:
#             gene_dictionary[row.Annotation_gene1] = {}
#         if row.Annotation_gene1 != row.Annotation_gene2:
#             gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

### For BUSCO tree
for row in df_filtered.itertuples():
    if (row.Gene1 not in to_remove) and (row.Gene2 not in to_remove):
        if row.Gene1 not in gene_dictionary:
            gene_dictionary[row.Gene1] = {}
        if row.Gene1 != row.Gene2:
            gene_dictionary[row.Gene1][row.Gene2] = row.Correlation


gene_dict = gene_dictionary
print("done")

import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))



for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)
            similarity_matrix[j, i] = 0


min_similarity = np.min(similarity_matrix)
print(min_similarity)

# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)


AA
AA
done
-0.67


In [11]:
import scipy.cluster.hierarchy as sch
from collections import defaultdict

Z_original = sch.linkage(condensed_distance_matrix, method="average")

num_bootstraps = 100  # Number of bootstrap resamples
bootstrap_support = defaultdict(int)

for num in range(num_bootstraps):
    print(num, end = " ")
    resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
    resampled_genes = [genes[i] for i in resampled_indices]
    
    resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
    condensed_resampled = squareform(resampled_distances)
    Z_resampled = sch.linkage(condensed_resampled, method="average")
    
    # Track clusters dynamically
    clusters = {i: {resampled_genes[i]} for i in range(len(resampled_genes))}
    
    for i in range(len(Z_resampled)):
        idx1 = int(Z_resampled[i, 0])
        idx2 = int(Z_resampled[i, 1])
        
        # Merge clusters
        new_cluster = clusters[idx1] | clusters[idx2]
        clusters[len(resampled_genes) + i] = new_cluster
        
        # Store cluster in bootstrap support
        bootstrap_support[frozenset(new_cluster)] += 1

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps)

# Function to create Newick format with bootstrap support
def generate_newick(Z, bootstrap_support, genes):
    def recurse(node):
        if node < len(genes):  # Leaf node
            return genes[int(node)]
        else:  # Internal node
            left = recurse(Z[int(node - len(genes)), 0])
            
            right = recurse(Z[int(node - len(genes)), 1])
            support = bootstrap_support.get(frozenset([left, right]), 0)
            
            return f"({left},{right}){':' + str(round(support, 2)) if support > 0 else ':0'}"

    # Start the recursive generation of the Newick string
    return recurse(len(Z) + len(genes) - 1) + ';'

# Generate Newick tree with bootstrap support
newick_tree = generate_newick(Z_original, bootstrap_support, genes)

# Print the Newick tree with bootstrap values
print("Newick Tree with Bootstrap Confidence:")
print(newick_tree)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 Newick Tree with Bootstrap Confidence:
(((((9108at7088,12709at7088):0.13,((6464at7088,860at7088):0.27,(1113at7088,3919at7088):0.01):0):0,((((13651at7088,(11719at7088,6598at7088):0.44):0,(8323at7088,(5074at7088,(4954at7088,11793at7088):0.37):0):0):0,((6414at7088,2703at7088):0.32,(((2258at7088,2750at7088):0.26,(6000at7088,7833at7088):0.19):0,(((8638at7088,9397at7088):0.2,(1505at7088,((2334at7088,5424at7088):0.11,((975at7088,5026at7088):0.16,(3834at7088,(8643at7088,((1939at7088,(6178at7088,5740at7088):0.37):0,(74at7088,2322at7088):0.39):0):0):0):0):0):0):0,((947at7088,8255at7088):0.34,(6987at7088,11090at7088):0.03):0):0):0):0):0,(((13315at7088,(14755at7088,4524at7088):0.29):0,((11383at7088,9292at7088):0.

In [9]:
# Define bootstrap threshold (e.g., 70% support)
bootstrap_threshold = 70

# Filter clusters based on threshold
filtered_bootstrap_support = {cluster: support for cluster, support in bootstrap_support.items() if support >= bootstrap_threshold}

# Generate Newick tree using only high-confidence clusters
newick_best_tree = generate_newick(Z_original, filtered_bootstrap_support, genes)

print("\nBest Newick Tree with High Bootstrap Support (≥ {}%):".format(bootstrap_threshold))
print(newick_best_tree)


Best Newick Tree with High Bootstrap Support (≥ 70%):
(((((((5144at7088,6703at7088):0,(9850at7088,(10188at7088,1946at7088):0):0):0,((10681at7088,14709at7088):0,((11303at7088,1907at7088):0,((4877at7088,9848at7088):0,(1686at7088,12472at7088):0):0):0):0):0,(((7361at7088,2982at7088):0,(13977at7088,11119at7088):0):0,((((3845at7088,9790at7088):0,(6330at7088,10529at7088):0):0,(4719at7088,(1955at7088,2465at7088):0):0):0,((14465at7088,(14955at7088,12028at7088):0):0,((5128at7088,13655at7088):0,((3231at7088,3067at7088):0,(1869at7088,10538at7088):0):0):0):0):0):0):0,((((3383at7088,(1551at7088,1360at7088):0):0,((4030at7088,10164at7088):0,(4680at7088,10459at7088):0):0):0,(((11058at7088,(14032at7088,8380at7088):0):0,((6115at7088,5464at7088):0,(15570at7088,10558at7088):0):0):0,(((4936at7088,4200at7088):0,(2357at7088,2754at7088):0):0,((7371at7088,11070at7088):0,(4779at7088,(1066at7088,4132at7088):0):0):0):0):0):0,((((12564at7088,(12613at7088,5916at7088):0):0,(7953at7088,(12895at7088,6381at7088):0):0):